In [2]:
# Import Dependencies
import pandas as pd
from datetime import datetime as dt


In [40]:
# read in hospital details csv files to a dataframe and format. Add columns for year and month for grouping
admit_details_df = pd.read_csv("data/Full_hospital_detail.csv")
admit_details_df = admit_details_df.rename(columns = {"Location": "hospital_code", "DOS": "date", 
    "Disposition": "disposition", "Decision to Discharge Time (min)": "decision_to_discharge_time"})
admit_details_df['date'] = pd.to_datetime(admit_details_df['date'], format="%m/%d/%Y")

#  Add columns for year and month for grouping. Reorder columns
admit_details_df['year'] = admit_details_df['date']
admit_details_df['year_month'] = admit_details_df['date']
admit_details_df['year'] = admit_details_df['year'].dt.strftime('%Y')
admit_details_df['year_month'] = admit_details_df['year_month'].dt.strftime('%Y-%m')
admit_details_df = admit_details_df[['hospital_code', 'year', 'year_month', 'date', 'disposition', 'decision_to_discharge_time']]
admit_details_df.head()

,hospital_code,year,year_month,date,disposition,decision_to_discharge_time
0,A,2021,2021-01,2021-01-01,Admitted,54.0
1,A,2021,2021-01,2021-01-01,Admitted,51.0
2,A,2021,2021-01,2021-01-01,Admitted,69.0
3,A,2021,2021-01,2021-01-01,Admit Obs,112.0
4,A,2021,2021-01,2021-01-01,Admitted,122.0


In [44]:
# Create data for month-by-month bar chart
# Wait time month-by-month for each hospital
wait_time_h = admit_details_df.groupby(['hospital_code', 'year_month'])['decision_to_discharge_time'].mean().reset_index()
wait_time_h = wait_time_h.rename(columns = {'decision_to_discharge_time':'mean_wait'})

In [46]:
# Wait time month-by-month for all hospitals
wait_time_a = admit_details_df.groupby(['year_month'])['decision_to_discharge_time'].mean().reset_index()
wait_time_a = wait_time_a.rename(columns = {'decision_to_discharge_time':'mean_wait'})
wait_time_a['hospital_code'] = 'all'
wait_time_a = wait_time_a[['hospital_code', 'year_month', 'mean_wait']]

In [48]:
# Concantenate wait time month-by-month dataframes into one dataframe for use in JS
wait_time_month = pd.concat([wait_time_h, wait_time_a], axis = 0)

In [52]:
# Create data for month-by-day wait times for each hospital
wait_time_day_h = admit_details_df.groupby(['hospital_code', 'year_month', 'date'])['decision_to_discharge_time']\
    .mean().reset_index()
wait_time_day_h = wait_time_day_h.rename(columns = {'decision_to_discharge_time':'mean_wait'})

In [53]:
# Create data for month-by-day wait times for all hospitals
wait_time_day_a = admit_details_df.groupby(['year_month', 'date'])['decision_to_discharge_time'].mean().reset_index()
wait_time_day_a = wait_time_day_a.rename(columns = {'decision_to_discharge_time':'mean_wait'})
wait_time_day_a['hospital_code'] = 'all'
wait_time_day_a = wait_time_day_a[['hospital_code', 'year_month', 'date', 'mean_wait']]

In [54]:
# Concantenate wait time month-by-month dataframes into one dataframe for use in JS
wait_time_day = pd.concat([wait_time_day_h, wait_time_day_a], axis = 0)

In [91]:
# Create data for pie chart
# Change Admit Obs to Admitted to simplify later code
admit_trans_detail = admit_details_df
admit_trans_detail['disposition'] = admit_trans_detail['disposition'].replace({'Admit Obs': "Admitted"})
admit_trans_detail.head()

,hospital_code,year,year_month,date,disposition,decision_to_discharge_time
0,A,2021,2021-01,2021-01-01,Admitted,54.0
1,A,2021,2021-01,2021-01-01,Admitted,51.0
2,A,2021,2021-01,2021-01-01,Admitted,69.0
3,A,2021,2021-01,2021-01-01,Admitted,112.0
4,A,2021,2021-01,2021-01-01,Admitted,122.0


In [93]:
# Create dataframe that groups by disposition and format to concantenate to other dataframes
disp_all = admit_trans_detail.groupby('disposition')['date'].count().reset_index()
disp_all = disp_all.rename(columns = {'date':"count"})
disp_all['hospital_code'] = 'all'
disp_all['year'] = '--'
disp_all = disp_all[['hospital_code', 'year', 'disposition', 'count']]
disp_all

,hospital_code,year,disposition,count
0,all,--,Admitted,227339
1,all,--,Transferred,20342


In [94]:
# Create dataframe that groups by hospital code and disposition with counts. Format to concantenate
disp_h = admit_trans_detail.groupby(['hospital_code','disposition'])['date'].count().reset_index()
disp_h = disp_h.rename(columns = {'date':"count"})
disp_h['year'] = '--'
disp_h = disp_h[['hospital_code', 'year', 'disposition', 'count']]
disp_h

,hospital_code,year,disposition,count
0,A,--,Admitted,14317
1,A,--,Transferred,1227
2,B,--,Admitted,3389
3,B,--,Transferred,1855
4,C,--,Admitted,2979
5,C,--,Transferred,1414
6,D,--,Admitted,35378
7,D,--,Transferred,623
8,E,--,Admitted,35382
9,E,--,Transferred,2401


In [99]:
# Create dataframe that groups by year for all hospitals and format to concantenate
disp_all_y = admit_trans_detail.groupby(['year','disposition'])['date'].count().reset_index()
disp_all_y = disp_all_y.rename(columns = {'date':"count"})
disp_all_y['hospital_code'] = 'all'
disp_all_y = disp_all_y[['hospital_code', 'year', 'disposition', 'count']]
disp_all_y

,hospital_code,year,disposition,count
0,all,2021,Admitted,113977
1,all,2021,Transferred,11177
2,all,2022,Admitted,111524
3,all,2022,Transferred,9057
4,all,2023,Admitted,1838
5,all,2023,Transferred,108


In [100]:
# Create dataframe that groups by hospital code and year and format to concantenate
disp_h_y = admit_trans_detail.groupby(['hospital_code', 'year', 'disposition'])['date'].count().reset_index()
disp_h_y = disp_h_y.rename(columns = {'date':"count"})
disp_h_y = disp_h_y[['hospital_code', 'year', 'disposition', 'count']]
disp_h_y

,hospital_code,year,disposition,count
0,A,2021,Admitted,7137
1,A,2021,Transferred,709
2,A,2022,Admitted,6995
3,A,2022,Transferred,515
4,A,2023,Admitted,185
...,...,...,...,...
61,K,2021,Transferred,327
62,K,2022,Admitted,16173
63,K,2022,Transferred,335
64,K,2023,Admitted,279


In [101]:
# Concantenate disposition dataframes into one dataframe to pass to JS
disposition = pd.concat([disp_all, disp_h, disp_all_y, disp_h_y], axis = 0)
disposition.head()

,hospital_code,year,disposition,count
0,all,--,Admitted,227339
1,all,--,Transferred,20342
0,A,--,Admitted,14317
1,A,--,Transferred,1227
2,B,--,Admitted,3389
